MODEL TRAINING

it is recommend to run the code on kaggle server in order to obtain best.pt file first but training it on local is also fine but way slower. If the best.pt file exist, the model will just use the weight from that without doing anything

In [5]:
import os
from pathlib import Path
import torch
from ultralytics import YOLO
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Check for GPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {DEVICE}")

# Define project structure (your local path)
base_dir = Path.cwd()
img_dir = base_dir / "images"
label_dir = base_dir / "labels"
output_dir = base_dir  # output in current directory
yaml_path = base_dir / "data.yaml"

# Class definitions (no 'person')
classes = ['helmet', 'vest', 'head']
num_classes = len(classes)

# Validate directories
for split in ['train', 'val', 'test']:
    if not (img_dir / split).exists() or not (label_dir / split).exists():
        logging.error(f"{split} directory for images or labels not found!")
        raise FileNotFoundError(f"Ensure 'images/{split}' and 'labels/{split}' exist in {base_dir}")

# Generate data.yaml file
with open(yaml_path, "w") as f:
    f.write(f"path: {base_dir.as_posix()}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("test: images/test\n")
    f.write(f"nc: {num_classes}\n")
    f.write("names:\n")
    for i, name in enumerate(classes):
        f.write(f"  {i}: {name}\n")
logging.info(f"Generated data.yaml at {yaml_path}")

# Generate hyp.yaml file for PPE-specific augmentation
hyp_path = base_dir / "hyp.yaml"
with open(hyp_path, "w") as f:
    f.write("lr0: 0.01\n")
    f.write("lr1: 0.02\n")
    f.write("momentum: 0.937\n")
    f.write("weight_decay: 0.0005\n")
    f.write("fl_gamma: 0.0\n")
    f.write("hsv_hue: 0.015\n")
    f.write("hsv_saturation: 0.7\n")
    f.write("hsv_value: 0.4\n")
    f.write("degrees: 5.0\n")
    f.write("translate: 0.1\n")
    f.write("scale: 0.5\n")
    f.write("shear: 0.0\n")
    f.write("flipud: 0.5\n")
    f.write("fliplr: 0.5\n")
    f.write("mosaic: 1.0\n")
    f.write("mixup: 0.0\n")
logging.info(f"Generated hyp.yaml for augmentation at {hyp_path}")

# Check for existing weights
weights_path = base_dir / "best.pt"
if weights_path.exists():
    model = YOLO(weights_path)
    logging.info(f"best.pt already exists at {weights_path}. Skipping training.")
else:
    logging.info("No best.pt found. Starting training from yolov8n.pt...")
    model = YOLO("yolov8n.pt")

    results = model.train(
        data=str(yaml_path),
        epochs=50,
        imgsz=640,
        batch=16,
        name="helmet_vest_detection",
        project="runs/train",
        device=DEVICE,
        augment=True
    )

    logging.info(f"Training completed. Weights saved to: {results.save_dir}/weights/best.pt")


2025-06-09 18:06:35,427 - INFO - Using device: cuda
2025-06-09 18:06:35,431 - INFO - Generated data.yaml at c:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest\data.yaml
2025-06-09 18:06:35,433 - INFO - Generated hyp.yaml for augmentation at c:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest\hyp.yaml
2025-06-09 18:06:35,884 - INFO - best.pt already exists at c:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest\best.pt. Skipping training.


Model testing on testing data

In [11]:
from matplotlib import pyplot as plt
import cv2
import logging
from pathlib import Path
import os
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import numpy as np

# Define paths
test_img_dir = base_dir / "images" / "test"
test_label_dir = base_dir / "labels" / "test"
save_dir = base_dir / "predictions"
save_dir.mkdir(parents=True, exist_ok=True)

# Define custom colors for each class
colors = {
    'helmet': (0, 255, 0),   # green
    'vest': (255, 255, 0),   # cyan/yellow
    'head': (0, 0, 255)      # red
}

# Store all predictions and labels
y_true = []
y_pred = []

# Ensure test image directory exists
if not test_img_dir.exists():
    logging.error(f"Test image directory not found: {test_img_dir}")
else:
    # Collect and sort all image files
    test_images = sorted([
        f for f in os.listdir(test_img_dir)
        if f.endswith(('.png', '.jpg', '.jpeg'))
    ])

    logging.info(f"Found {len(test_images)} test images.")

    # Run prediction, save annotated images, and collect labels
    for img_name in test_images:
        img_path = test_img_dir / img_name
        label_file = test_label_dir / f"{img_name.rsplit('.', 1)[0]}.txt"

        # Predict
        results = model.predict(source=str(img_path), conf=0.25, save=False)

        for r in results:
            # Collect predicted classes
            pred_classes = r.boxes.cls.cpu().numpy().astype(int).tolist()

            # Read ground truth labels
            if label_file.exists():
                with open(label_file) as f:
                    true_classes = [int(line.strip().split()[0]) for line in f.readlines()]
            else:
                logging.warning(f"Label file missing for {img_name}")
                true_classes = []

            # Only add if counts match
            if len(true_classes) == len(pred_classes):
                y_true.extend(true_classes)
                y_pred.extend(pred_classes)
            else:
                logging.warning(
                    f"Label-prediction mismatch in {img_name}: "
                    f"{len(true_classes)} ground truth vs {len(pred_classes)} predicted"
                )

            # Load original image for drawing
            img_annotated = cv2.imread(str(img_path))

            # Draw boxes with custom colors
            for box in r.boxes:
                cls_id = int(box.cls[0])
                conf = float(box.conf[0])
                xyxy = box.xyxy[0].cpu().numpy().astype(int)
                x1, y1, x2, y2 = xyxy
                cls_name = model.names[cls_id]
                color = colors.get(cls_name, (255, 255, 255))  # default = white
                label = f"{cls_name} {conf:.2f}"

                cv2.rectangle(img_annotated, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img_annotated, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # Save the annotated image
            save_path = save_dir / img_name
            cv2.imwrite(str(save_path), img_annotated)

    logging.info(f"✅ Saved all annotated images to: {save_dir}")

2025-06-09 18:14:31,961 - INFO - Found 2455 test images.



image 1/1 c:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest\images\test\000009.jpg: 640x640 1 helmet, 1 vest, 25.5ms
Speed: 3.5ms preprocess, 25.5ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)


2025-06-09 18:14:32,007 - WARNING - Label-prediction mismatch in 000009.jpg: 1 ground truth vs 2 predicted



image 1/1 c:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest\images\test\000010.jpg: 640x640 5 helmets, 5 vests, 24.5ms
Speed: 3.0ms preprocess, 24.5ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)


2025-06-09 18:14:32,050 - WARNING - Label-prediction mismatch in 000010.jpg: 8 ground truth vs 10 predicted


...

2025-06-09 18:15:50,519 - WARNING - Label-prediction mismatch in pos_897.jpg: 3 ground truth vs 4 predicted



image 1/1 c:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest\images\test\pos_908.jpg: 640x640 1 helmet, 8.0ms
Speed: 2.1ms preprocess, 8.0ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest\images\test\pos_925.jpg: 640x640 3 helmets, 2 vests, 7.8ms
Speed: 2.2ms preprocess, 7.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)


2025-06-09 18:15:50,564 - WARNING - Label-prediction mismatch in pos_925.jpg: 4 ground truth vs 5 predicted



image 1/1 c:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest\images\test\pos_927.jpg: 640x640 4 helmets, 7.8ms
Speed: 2.1ms preprocess, 7.8ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest\images\test\pos_93.jpg: 640x640 1 vest, 7.8ms
Speed: 2.3ms preprocess, 7.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest\images\test\pos_950.jpg: 640x640 2 helmets, 7.9ms
Speed: 2.4ms preprocess, 7.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest\images\test\pos_962.jpg: 640x640 1 helmet, 9.5ms
Speed: 2.2ms preprocess, 9.5ms inference, 1.3

2025-06-09 18:15:50,705 - INFO - ✅ Saved all annotated images to: c:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest\predictions


Obtain Testing Data mAP50 and mAP50-95 by Class

In [12]:
results = model.val(
    data=str(yaml_path),
    split="test"
)



Ultralytics 8.3.137  Python-3.13.2 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
val: Fast image access  (ping: 0.10.1 ms, read: 480.9144.9 MB/s, size: 243.6 KB)


val: Scanning C:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest\labels\test.cache... 2455 images, 2 backgrounds, 0 corrupt: 100%|██████████| 2455/2455 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 154/154 [00:20<00:00,  7.40it/s]


                   all       2455      20193      0.884      0.842      0.894      0.561
                helmet       1822       6749      0.939      0.857      0.918      0.625
                  vest        440        935      0.811      0.772      0.854      0.562
                  head        713      12509      0.902      0.896      0.911      0.497
Speed: 0.3ms preprocess, 3.4ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\val7


Visualize Confusion Matrix in Code

In [13]:
# Confusion Matrix
if y_true and y_pred:
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(classes))))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    plt.figure(figsize=(8, 6))
    disp.plot(cmap="Blues", values_format='d')
    plt.title("Confusion Matrix")
    plt.show()

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=classes))
else:
    logging.error("No valid prediction-label pairs to compute metrics.")

<Figure size 800x600 with 0 Axes>

<Figure size 640x480 with 2 Axes>


Classification Report:
              precision    recall  f1-score   support

      helmet       0.94      0.90      0.92      3369
        vest       0.75      0.76      0.76       475
        head       0.92      0.97      0.95      2616

    accuracy                           0.92      6460
   macro avg       0.87      0.88      0.88      6460
weighted avg       0.92      0.92      0.92      6460



Evaluation of Testing data detection performance

In [14]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Define output files (with filenames)
conf_matrix_path = output_dir / "confusion_matrix.png"
cls_report_path = output_dir / "classification_report.txt"
pred_dist_path = output_dir / "prediction_distribution.png"

# Run evaluation only if predictions exist
if y_true and y_pred:
    # Convert to numpy arrays
    y_true_np = np.array(y_true)
    y_pred_np = np.array(y_pred)

    # Overall Accuracy
    acc = accuracy_score(y_true_np, y_pred_np)
    print(f"\n✅ Overall Accuracy: {acc:.4f}")

    # Per-class Accuracy
    per_class_acc = {}
    for i, class_name in enumerate(classes):
        true_mask = y_true_np == i
        if true_mask.sum() > 0:
            acc_i = (y_pred_np[true_mask] == i).sum() / true_mask.sum()
            per_class_acc[class_name] = acc_i
        else:
            per_class_acc[class_name] = None  # No samples in ground truth

    print("\nPer-Class Accuracy:")
    for cls, acc_i in per_class_acc.items():
        if acc_i is not None:
            print(f"  {cls}: {acc_i:.4f}")
        else:
            print(f"  {cls}: No true instances in test set")

    # Confusion Matrix Plot
    cm = confusion_matrix(y_true_np, y_pred_np, labels=list(range(len(classes))))
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap="Blues",
                xticklabels=classes, yticklabels=classes,
                annot_kws={"size": 16})
    plt.title("Confusion Matrix", fontsize=16)
    plt.xlabel("Predicted", fontsize=14)
    plt.ylabel("True", fontsize=14)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.tight_layout()
    plt.savefig(conf_matrix_path)
    plt.close()
    print(f"✅ Saved confusion matrix to: {conf_matrix_path.name}")

    # Classification Report
    report = classification_report(y_true_np, y_pred_np, target_names=classes)
    print("\nClassification Report:")
    print(report)
    with open(cls_report_path, "w") as f:
        f.write(f"Accuracy: {acc:.4f}\n\n")
        f.write("Per-Class Accuracy:\n")
        for cls, acc_i in per_class_acc.items():
            if acc_i is not None:
                f.write(f"  {cls}: {acc_i:.4f}\n")
            else:
                f.write(f"  {cls}: No true instances in test set\n")
        f.write("\n" + report)
    print(f"✅ Saved classification report to: {cls_report_path.name}")

    # Prediction Distribution Histogram
    plt.figure(figsize=(6, 4))
    sns.histplot(y_pred_np, bins=np.arange(len(classes) + 1) - 0.5, kde=False)
    plt.xticks(np.arange(len(classes)), classes, fontsize=12)
    plt.yticks(fontsize=12)
    plt.title("Prediction Distribution", fontsize=16)
    plt.xlabel("Predicted Class", fontsize=14)
    plt.ylabel("Frequency", fontsize=14)
    plt.tight_layout()
    plt.savefig(pred_dist_path)
    plt.close()
    print(f"✅ Saved prediction distribution to: {pred_dist_path.name}")

else:
    logging.error("❌ No valid prediction-label pairs to compute metrics.")



✅ Overall Accuracy: 0.9204

Per-Class Accuracy:
  helmet: 0.9032
  vest: 0.7621
  head: 0.9713
✅ Saved confusion matrix to: confusion_matrix.png

Classification Report:
              precision    recall  f1-score   support

      helmet       0.94      0.90      0.92      3369
        vest       0.75      0.76      0.76       475
        head       0.92      0.97      0.95      2616

    accuracy                           0.92      6460
   macro avg       0.87      0.88      0.88      6460
weighted avg       0.92      0.92      0.92      6460

✅ Saved classification report to: classification_report.txt
✅ Saved prediction distribution to: prediction_distribution.png


Model Training Results Evaluation Plot and Graph

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Path to CSV and output directory
results_csv = base_dir / "results2.csv"

# Load CSV
df = pd.read_csv(results_csv)

# Set seaborn style
sns.set(style="whitegrid")

# Plot 1: mAP50, mAP50-95, and Box Loss
plt.figure(figsize=(12, 8))
plt.plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP50', color='blue')
plt.plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP50-95', color='green')
plt.plot(df['epoch'], df['train/box_loss'], label='Train Box Loss', color='red')
plt.plot(df['epoch'], df['val/box_loss'], label='Val Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=20)
plt.ylabel('Score / Loss', fontsize=20)
plt.title('mAP & Box Loss Over Epochs', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20)
plt.grid(True)
plt.tight_layout()
plt.savefig(output_dir / 'metrics_map_boxloss.png')
plt.close()

# Plot 2: Class Loss and DFL Loss (Train vs Val)
plt.figure(figsize=(12, 8))
plt.plot(df['epoch'], df['train/cls_loss'], label='Train Class Loss', color='purple')
plt.plot(df['epoch'], df['val/cls_loss'], label='Val Class Loss', color='brown')
plt.plot(df['epoch'], df['train/dfl_loss'], label='Train DFL Loss', color='teal')
plt.plot(df['epoch'], df['val/dfl_loss'], label='Val DFL Loss', color='gray')
plt.xlabel('Epoch', fontsize=20)
plt.ylabel('Loss', fontsize=20)
plt.title('Classification & DFL Loss Over Epochs', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20)
plt.grid(True)
plt.tight_layout()
plt.savefig(output_dir / 'metrics_cls_dfl_loss.png')
plt.close()

print(f"✅ Saved plots to: {output_dir}")


✅ Saved plots to: c:\WORK STUFF LOL\NTU file\NTU file\NTU courses\3rd year\Deep learning\Individual Project3\Hard Hat - Vest


Real Time Detection

In [18]:
import cv2
import time
import torch
from ultralytics import YOLO

# Load trained YOLO model
model = YOLO(weights_path)  # replace with your trained weights

# Define class-specific confidence thresholds
class_thresholds = {
    'helmet': 0.4,
    'vest': 0.25,
    'head': 0
}

# Define colors for each class
colors = {
    'helmet': (0, 255, 0),       # green
    'vest': (255, 255, 0),       # light blue (cyan-yellow mix)
    'head': (0, 0, 255)          # red
}

# Load video (0 for webcam)
video_path = 0
cap = cv2.VideoCapture(video_path)

# Output video setup
output_path = "webcam_video.mp4"
fps = cap.get(cv2.CAP_PROP_FPS) or 30
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# FPS tracking
frame_count = 0
start_time = time.time()

# Start detection loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Inference
    results = model.predict(source=frame, conf=0.05, iou=0.5, verbose=False)  # low base conf to allow manual filtering

    annotated_frame = frame.copy()

    for result in results:
        for box in result.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            xyxy = box.xyxy[0].cpu().numpy().astype(int)
            x1, y1, x2, y2 = xyxy
            cls_name = model.names[cls_id]
            threshold = class_thresholds.get(cls_name, 0.25)

            if conf >= threshold:
                color = colors.get(cls_name, (255, 255, 255))  # fallback: white
                label = f"{cls_name} {conf:.2f}"
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(annotated_frame, label, (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Write to output and optionally show
    out.write(annotated_frame)
    cv2.imshow("YOLOv8 Video", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# FPS calculation
end_time = time.time()
elapsed_time = end_time - start_time
inference_fps = frame_count / elapsed_time

print(f"\n📊 Total frames: {frame_count}")
print(f"⏱️ Elapsed time: {elapsed_time:.2f} seconds")
print(f"⚡ Inference FPS: {inference_fps:.2f}")

# Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()



📊 Total frames: 779
⏱️ Elapsed time: 26.81 seconds
⚡ Inference FPS: 29.06


Video FPS Check (Before Detection)

In [45]:
import cv2
import time

# Load video (or use 0 for webcam)
video_path = "Video1.mp4"  # Change to 0 for webcam
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("❌ Error opening video file.")
    exit()

frame_count = 0
start_time = time.time()

# Read frames and count
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

cap.release()
end_time = time.time()

# Calculate and print FPS
elapsed_time = end_time - start_time
fps = frame_count / elapsed_time

print(f"📊 Total Frames: {frame_count}")
print(f"⏱️ Total Time: {elapsed_time:.2f} seconds")
print(f"⚡ Raw Video FPS (read speed only): {fps:.2f}")


📊 Total Frames: 909
⏱️ Total Time: 4.21 seconds
⚡ Raw Video FPS (read speed only): 215.67
